In [96]:
pip install geopy

     |████████████████████████████████| 119 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 5.6 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [495]:
import pandas as pd
import geopy.distance
import numpy as np

In [496]:
counties = pd.read_csv('uscounties.csv')

In [497]:
counties.describe()

,county_fips,lat,lng,population
count,3143.000000,3143.000000,3143.000000,3.143000e+03
mean,30374.576519,38.463189,-92.284373,1.049079e+05
std,15168.624870,5.322459,12.892769,3.342727e+05
min,1001.000000,19.598500,-164.027200,4.800000e+01
25%,18176.000000,34.696250,-98.239250,1.083600e+04
50%,29175.000000,38.385200,-90.404200,2.580800e+04
75%,45080.000000,41.811050,-83.441700,6.782450e+04
max,56045.000000,69.312200,-67.628700,1.001964e+07


In [498]:
total_pop = counties['population'].sum()
total_pop

329725481

In [499]:
n_clusters = 68
cluster_pop = round(total_pop/n_clusters)

In [589]:
min_threshold = round(cluster_pop*.9)
max_threshold = round(cluster_pop*1.1)
print(min_threshold, max_threshold)

4364014 5333794


In [501]:
counties['point'] = counties.apply(lambda x: (x['lat'], x['lng']), axis=1)

In [584]:
counties.head()

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,point,starting_dist,cluster
0,Montgomery,Montgomery,Montgomery County,51121,VA,Virginia,37.1743,-80.3872,99294,"(37.1743, -80.3872)",0.000000,34
1,Radford,Radford,Radford City,51750,VA,Virginia,37.1229,-80.5587,16205,"(37.1229, -80.5587)",10.108850,34
2,Floyd,Floyd,Floyd County,51063,VA,Virginia,36.9316,-80.3625,15536,"(36.9316, -80.3625)",16.791919,34
3,Roanoke,Roanoke,Roanoke County,51161,VA,Virginia,37.2693,-80.0680,96303,"(37.2693, -80.068)",18.782850,34
4,Pulaski,Pulaski,Pulaski County,51155,VA,Virginia,37.0635,-80.7144,33893,"(37.0635, -80.7144)",19.618061,46


In [586]:
counties[counties['county_fips']==20183]

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,point,starting_dist,cluster
2271,Smith,Smith,Smith County,20183,KS,Kansas,39.7852,-98.7855,3588,"(39.7852, -98.7855)",1011.775577,13


In [583]:
def state_maker(counties, starting_county = counties.sample(n=1)):
    starting_point = (starting_county.iloc[0]['lat'], starting_county.iloc[0]['lng'])
    counties['starting_dist'] = counties.apply(lambda x: geopy.distance.distance(x['point'], starting_point).miles, axis=1)
    counties = counties.sort_values(by='starting_dist').reset_index(drop=True)
    counties['cluster'] = -1
    k = 0
    for index in counties.index:
        row = counties.iloc[index]
        temp_cluster_pop = 0
        c_id = row['county_fips']
        if len(counties[counties['cluster']==-1])<1:
            break
        elif row['cluster'] > -1:
            continue
        elif row['population'] > max_threshold:
            print(row['county'], k)
            counties.loc[counties['county_fips']==c_id, 'cluster'] = k
            k+=1
            continue
        else:
            copy = counties[counties['cluster']==-1].copy()
            copy_city = copy.loc[copy['county_fips']==c_id]
            copy['current_dist'] = copy.apply(lambda x: geopy.distance.distance(x['point'], copy_city['point']).miles, axis=1)
            copy = copy.sort_values(by='current_dist')
            for i, r in copy.iterrows():
                if temp_cluster_pop + r['population'] > max_threshold:
                    continue
                else:
                    copy_id = r['county_fips']
                    counties.loc[counties['county_fips']==copy_id, 'cluster'] = k
                    temp_cluster_pop += r['population']
                    if temp_cluster_pop >= min_threshold:
                        k+=1
                        break
    return counties

In [587]:
counties_v2 = state_maker(counties, counties[counties['county_fips']==20183])

Los Angeles 51


In [590]:
counties_v2.groupby('cluster')['population'].sum().describe()

count    7.100000e+01
mean     4.644021e+06
std      7.424757e+05
min      2.304107e+06
25%      4.392714e+06
50%      4.515783e+06
75%      4.713864e+06
max      1.001964e+07
Name: population, dtype: float64

In [588]:
counties_v2.to_csv('uscounties_clustered.csv')

In [524]:
# assume we have a list of locations called "locations"
# and a function called "run_experiment" that runs one experiment
# and returns the clusters as a list of lists of locations
locations = counties['county_fips']
num_locations = len(locations)
freq_matrix = pd.DataFrame(np.zeros((num_locations, num_locations)), 
                           index=locations, columns=locations)

for x in range(50):
    # run one experiment
    counties = state_maker(counties)
    for fip in counties['county_fips']:
        temp_k = counties.loc[counties['county_fips']==fip, 'cluster'].values[0]
        neighbors = counties.loc[counties['cluster']==temp_k, 'county_fips'].values.tolist()
        for neighbor in neighbors:
            freq_matrix.at[fip, neighbor] += 1

# divide by 50 to get frequencies
freq_matrix /= 50

Los Angeles 66
Los Angeles 66
Los Angeles 63
Los Angeles 46
Los Angeles 64
Los Angeles 65
Los Angeles 65
Los Angeles 65
Los Angeles 5
Los Angeles 65
Los Angeles 64
Los Angeles 5
Los Angeles 7
Los Angeles 66
Los Angeles 64
Los Angeles 36
Los Angeles 65
Los Angeles 65
Los Angeles 65
Los Angeles 66
Los Angeles 68
Los Angeles 64
Los Angeles 65
Los Angeles 68
Los Angeles 67
Los Angeles 68
Los Angeles 65
Los Angeles 34
Los Angeles 10
Los Angeles 64
Los Angeles 65
Los Angeles 65
Los Angeles 16
Los Angeles 64
Los Angeles 68
Los Angeles 10
Los Angeles 66
Los Angeles 65
Los Angeles 65
Los Angeles 14
Los Angeles 68
Los Angeles 67
Los Angeles 63
Los Angeles 68
Los Angeles 65
Los Angeles 66
Los Angeles 69
Los Angeles 49
Los Angeles 10
Los Angeles 65


In [525]:
freq_matrix.sort_index(inplace=True)
freq_matrix.head()

county_fips,16037,16059,16013,16025,16023,16085,16015,16039,16063,16047,...,12087,12086,2188,2158,2180,15009,15005,15001,15003,15007
county_fips,,,,,,,,,,,,,,,,,,,,,
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [535]:
freq_matrix.loc[1001, 1001]

1.0

In [562]:
neighbors = freq_matrix.loc[i, freq_matrix.loc[i]>.5]
for neighbor in neighbors.index:
    print(neighbor)

56003
56043
30003
56033
56019
56025
30087
56007
30075
56005
30017
56009
30079
56001
56011
56045
30011
56031
56027
30021
30025
8069
46081
30109
56015
30083
56021
46019
46063
38033
38011
46033
46047
31165
38087
31157
38007
8123
31007
31045
46103
46093
38053
31105
31013
46105
38001
38105
46102
38089
38041
31123
38025
31161
31033
46137
46071
46055
31069
38037
38061
46007
38057
38013
46031
38059
38065
38085
46041
38055
38101
46117
38075
38029
46119
46129
46021
46107
38049
38083
38009
46089


In [580]:
counties['cluster'] = -1
k=0
for i in freq_matrix.index:
    if counties.loc[counties['county_fips']==i, 'cluster'].values[0] !=-1:
        continue
    else:
        neighbors = freq_matrix.loc[i, freq_matrix.loc[i]>.1]
        for neighbor in neighbors.index:
            if counties.loc[counties['county_fips']==neighbor, 'cluster'].values[0] !=-1:
                continue
            else:
                counties.loc[counties['county_fips']==neighbor, 'cluster'] = k
        k+=1

In [581]:
counties['cluster'].value_counts()[-20:]

15    16
47    15
42    14
41    11
35    10
11     8
40     8
39     6
48     5
52     4
20     4
50     2
45     1
56     1
5      1
38     1
24     1
12     1
10     1
55     1
Name: cluster, dtype: int64

In [582]:
counties.groupby('cluster').sum()['population'].describe()

count    5.700000e+01
mean     5.784658e+06
std      3.431399e+06
min      1.194180e+05
25%      3.233665e+06
50%      5.387770e+06
75%      8.144395e+06
max      1.290038e+07
Name: population, dtype: float64

In [579]:
counties.to_csv('uscounties_clustered_meta.csv')